In [1]:
pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 8.4 MB/s 
     |████████████████████████████████| 240 kB 28.7 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=849c8c439900f7092e6e53633b607e360582eed336ca7e30ce175b2a482f4f54
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


In [2]:
import argparse
import spacy
#import unidecode
import re
import nltk
#import contractions

import pandas as pd
import numpy as np 

from nltk import word_tokenize, sent_tokenize

import os
from google.colab import drive


from gensim.models import word2vec, FastText


from matplotlib import pyplot as plt
import plotly.graph_objects as go


nltk.download('punkt')
# load spacy model, can be "en_core_web_sm" as well
#nlp = spacy.load('en_core_web_md')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Define intersection between attackt example functions 

In [3]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
   
def remove_brackets(sample_str):
    sample_str = sample_str.replace("[[", "")
    sample_str = sample_str.replace("]]", "")
    return sample_str

def clean_up_list(str_list):
    final_list=[]
    for item in str_list:
        final_list.append(remove_brackets(item))
    return final_list

def comparing(lst1, lst2):
    items = intersection(lst1, lst2)
    return len(items), items

# Generate a report from the imtersection between 2 different arracks
def intersection_report(failed_lst1, failed_lst2,succeeded_lst1, succeeded_lst2, attack1, attack2):
    
    f_intersect_num, f_intersect_items = comparing(failed_lst1, failed_lst2)
    print('Intersection number of failed examples between', attack1, 'and', attack2, sep=" ")
    print(f_intersect_num)
   
    s_intersect_num, s_intersect_items = comparing(succeeded_lst1, succeeded_lst2)
    print('Intersection number of succeeded examples between', attack1, 'and', attack2, sep=" ")
    print(s_intersect_num)
    
    return f_intersect_items, s_intersect_items

# Parse textattack results
def parse_text_file(filename):
    # Constant Variable
    divide_section_string = '--------------------------------------------- Result'
    # Initialize variable
    count = 0
    failed_examples = []
    succeeded_examples = []
    attack_examples = []
    # Read text file line by line
    f = open(filename, 'r')
    lines = f.readlines()
   
    pattern = re.compile(r'\[\[[0-9a-zA-Z\'\-]+\]\]')

    while count < len(lines):
        if divide_section_string in lines[count]: 
            count += 1
            if '[[[FAILED]]]' in lines[count] or '[[SKIPPED]]' in lines[count]:
                count += 2
                failed_examples.append(lines[count])
            else:
                count += 2
                succeeded_examples.append(lines[count])
                count += 2
                attack_examples.append(lines[count])
            count +=1
        count += 1       

    return failed_examples, succeeded_examples, attack_examples

### Extract purturbed words from attack examples 

In [4]:
def intersection_with_index(lst1, lst2):
    new_lst = []
    index1 = []
    index2 = []
    no_insec_index1 = []
    no_insec_index2 = []
    for item in lst2:
        if item in lst1:
            new_lst.append(item)
            index1.append(lst1.index(item))
            index2.append(lst2.index(item))
        else:
            #no_insec_index1.append(lst1.index(item))
            no_insec_index2.append(lst2.index(item))
    
    for item in lst1:
        if item not in lst2:
            no_insec_index1.append(lst1.index(item))  
    
    return new_lst, index1, index2

def remove_bracket_square_word(word):
    word = word[2:len(word)-2]
    return word

def extract_purturb_words(lst, indexs):
    match_lst = []
    pattern = re.compile(r'\[\[[0-9a-zA-Z\'\-]+\]\]')
    for index in indexs:
        matches = pattern.findall(lst[index])
        match_lst.append([remove_bracket_square_word(word) for word in matches])
    return match_lst

In [5]:
def difference(lst1, lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    return lst3

def extract_intesect_purturb_words(clean_s_examples1, clean_s_examples2, s_examples1, s_examples2, attack_examples1, attack_examples2):
    new_lst, index1, index2 = intersection_with_index(clean_s_examples1, clean_s_examples2)
    purturbed_words1 = extract_purturb_words(s_examples1, index1)
    purturbed_words2 = extract_purturb_words(s_examples2, index2)
    replaced_words1 = extract_purturb_words(attack_examples1, index1)
    replaced_words2 = extract_purturb_words(attack_examples2, index2)
    
    intersect_purturbed_words = []
    for i in range(len(purturbed_words1)):
        intersect_purturbed_words.append(intersection(purturbed_words1[i], purturbed_words2[i]))
    
    not_in_2 = []
    for i in range(len(purturbed_words1)):
        not_in_2.append(difference(purturbed_words1[i], purturbed_words2[i]))
    not_in_1 = []
    for i in range(len(purturbed_words1)):
        not_in_1.append(difference(purturbed_words2[i], purturbed_words1[i]))
        
    return intersect_purturbed_words, index1, index2, not_in_1, not_in_2

In [6]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


### Import textattack results from performing different attack method

In [7]:
t_failed_examples, t_succeeded_examples, t_attack_examples = parse_text_file('/content/gdrive/MyDrive/albert_textfooler')
d_failed_examples, d_succeeded_examples, d_attack_examples = parse_text_file('/content/gdrive/MyDrive/albert_deepwordbug')
p_failed_examples, p_succeeded_examples, p_attack_examples = parse_text_file('/content/gdrive/MyDrive/albert_pwws')
b_failed_examples, b_succeeded_examples, b_attack_examples = parse_text_file('/content/gdrive/MyDrive/albert_bae')


t_400_failed_examples, t_400_succeeded_examples, t_400_attack_examples = parse_text_file('/content/gdrive/MyDrive/400_textfooler_albert')
p_400_failed_examples, p_400_succeeded_examples, p_400_attack_examples = parse_text_file('/content/gdrive/MyDrive/400_pwws_albert')
d_400_failed_examples, d_400_succeeded_examples, d_400_attack_examples = parse_text_file('/content/gdrive/MyDrive/400_deepwordbug_albert')
b_400_failed_examples, b_400_succeeded_examples, b_400_attack_examples = parse_text_file('/content/gdrive/MyDrive/400_bae_albert')


t_s = clean_up_list(t_succeeded_examples) + clean_up_list(t_400_succeeded_examples)
p_s = clean_up_list(p_succeeded_examples) + clean_up_list(p_400_succeeded_examples)
d_s = clean_up_list(d_succeeded_examples) + clean_up_list(d_400_succeeded_examples)
b_s = clean_up_list(b_succeeded_examples) + clean_up_list(b_400_succeeded_examples)

t_a = clean_up_list(t_attack_examples) + clean_up_list(t_400_attack_examples)
p_a = clean_up_list(p_attack_examples) + clean_up_list(p_400_attack_examples)
d_a = clean_up_list(d_attack_examples) + clean_up_list(d_400_attack_examples)
b_a = clean_up_list(b_attack_examples) + clean_up_list(b_400_attack_examples)

t_succeeded = t_succeeded_examples + t_400_succeeded_examples
p_succeeded = p_succeeded_examples + p_400_succeeded_examples
d_succeeded = d_succeeded_examples + d_400_succeeded_examples
b_succeeded = b_succeeded_examples + b_400_succeeded_examples

t_attack = t_attack_examples + t_400_attack_examples
p_attack = p_attack_examples + p_400_attack_examples
d_attack = d_attack_examples + d_400_attack_examples
b_attack = b_attack_examples + b_400_attack_examples



# del d_s[36]
# del d_a[36]
# del d_succeeded[36]
# del d_attack[36]

In [8]:
df_test = pd.read_csv('/content/gdrive/MyDrive/data/test_emails.csv')

In [9]:
def count_phish_legit_attack(s_examples,succeed_examples,attack_examples,df):
    count_phish = 0
    count_legit = 0
    s_phish = []
    a_phish = []
    s_legit = []
    a_legit = []
    indexes = []
    lst = df['text'].tolist()
    label_lst = df['label'].tolist()
    for i in range(len(s_examples)):
        #print(i)
        #indexes.append(lst.index(s_examples[i].replace('\n', '')))
        index = lst.index(s_examples[i].replace('\n', ''))
        if df.iloc[index]['label'] == 1:
            count_phish += 1
            s_phish.append(succeed_examples[i])
            a_phish.append(attack_examples[i])
        else:
            count_legit += 1
            s_legit.append(succeed_examples[i])
            a_legit.append(attack_examples[i])
                          
    print('Phishing: ' + str(count_phish) + ' Legitimate: ' + str(count_legit))
    return s_phish, a_phish, s_legit, a_legit 

### Textfooler

In [10]:
t_s_phish, t_a_phish, t_s_legit, t_a_legit = count_phish_legit_attack(t_s,t_succeeded,t_attack,df_test)

Phishing: 52 Legitimate: 60


### PWWS

In [11]:
p_s_phish, p_a_phish, p_s_legit, p_a_legit = count_phish_legit_attack(p_s,p_succeeded,p_attack,df_test)

Phishing: 52 Legitimate: 18


### DeepWordBug

In [12]:
d_s_phish, d_a_phish, d_s_legit, d_a_legit = count_phish_legit_attack(d_s,d_succeeded,d_attack,df_test)

Phishing: 52 Legitimate: 14


### BAE

In [13]:
b_s_phish, b_a_phish, b_s_legit, b_a_legit = count_phish_legit_attack(b_s,b_succeeded,b_attack,df_test)

Phishing: 7 Legitimate: 20


### Find the intersection between different attack results

In [14]:
print('Intersection between textfooler and pwws')
count_t_p, lst_t_p = comparing(t_s,p_s)
print(count_t_p)
print('phishing')
count_t_p_phish, lst_t_p_phish = comparing(clean_up_list(t_s_phish),clean_up_list(p_s_phish))
print(count_t_p_phish)
print('legit')
count_t_p_legit, lst_t_p_legit = comparing(clean_up_list(t_s_legit),clean_up_list(p_s_legit))
print(count_t_p_legit)

print('Intersection between textfooler and deepwordbug')
count_t_d, lst_t_d = comparing(t_s,d_s)
print(count_t_d)
print('phishing')
count_t_d_phish, lst_t_d_phish = comparing(clean_up_list(t_s_phish),clean_up_list(d_s_phish))
print(count_t_d_phish)
print('legit')
count_t_d_legit, lst_t_d_legit = comparing(clean_up_list(t_s_legit),clean_up_list(d_s_legit))
print(count_t_d_legit)

print('Intersection between textfooler and bae')
count_t_b, lst_t_b = comparing(t_s,b_s)
print(count_t_b)
print('phishing')
count_t_b_phish, lst_t_b_phish = comparing(clean_up_list(t_s_phish),clean_up_list(b_s_phish))
print(count_t_b_phish)
print('legit')
count_t_b_legit, lst_t_p_legit = comparing(clean_up_list(t_s_legit),clean_up_list(b_s_legit))
print(count_t_b_legit)

print('Intersection between pwws and deepwordbug')
count_p_d, lst_p_d = comparing(p_s,d_s)
print(count_p_d)

print('phishing')
count_p_d_phish, lst_p_d_phish = comparing(clean_up_list(p_s_phish),clean_up_list(d_s_phish))
print(count_p_d_phish)
print('legit')
count_p_d_legit, lst_p_d_legit = comparing(clean_up_list(p_s_legit),clean_up_list(d_s_legit))
print(count_p_d_legit)

print('Intersection between pwws and bae')
count_p_b, lst_p_d = comparing(p_s,b_s)
print(count_p_b)

print('phishing')
count_p_b_phish, lst_p_b_phish = comparing(clean_up_list(p_s_phish),clean_up_list(b_s_phish))
print(count_p_b_phish)
print('legit')
count_p_b_legit, lst_p_b_legit = comparing(clean_up_list(p_s_legit),clean_up_list(b_s_legit))
print(count_p_b_legit)

print('Intersection between deepwordbug and bae')
count_d_b, lst_d_d = comparing(d_s,b_s)
print(count_d_b)

print('phishing')
count_d_b_phish, lst_d_b_phish = comparing(clean_up_list(d_s_phish),clean_up_list(b_s_phish))
print(count_d_b_phish)
print('legit')
count_d_b_legit, lst_d_b_legit = comparing(clean_up_list(d_s_legit),clean_up_list(b_s_legit))
print(count_d_b_legit)

print('************************************')
print('Intersection between textfooler, pwws and deepwordbug')
count_t_p_d, lst_t_p_d = comparing(d_s,lst_t_p)
print(count_t_p_d)

print('phishing')
count_t_p_d_phish, lst_t_p_d_phish = comparing(clean_up_list(d_s_phish),lst_t_p_phish)
print(count_t_p_d_phish)
print('legit')
count_t_p_d_legit, lst_t_p_d_legit = comparing(clean_up_list(d_s_legit),lst_t_p_legit)
print(count_t_p_d_legit)

print('Intersection between textfooler, pwws and bae')
count_t_p_b, lst_t_p_b = comparing(b_s,lst_t_p)
print(count_t_p_b)
print('phishing')
count_t_p_b_phish, lst_t_p_d_phish = comparing(clean_up_list(b_s_phish),lst_t_p_phish)
print(count_t_p_b_phish)
print('legit')
count_t_p_b_legit, lst_t_p_b_legit = comparing(clean_up_list(b_s_legit),lst_t_p_legit)
print(count_t_p_b_legit)


print('Intersection between textfooler, deepwordbug and bae')
count_t_d_b, lst_t_d_b = comparing(b_s,lst_t_d)
print(count_t_d_b)
print('phishing')
count_t_d_b_phish, lst_t_d_b_phish = comparing(clean_up_list(b_s_phish),lst_t_d_phish)
print(count_t_d_b_phish)
print('legit')
count_t_d_b_legit, lst_t_d_b_legit = comparing(clean_up_list(b_s_legit),lst_t_d_legit)
print(count_t_d_b_legit)




print('Intersection between pwws, deepwordbug and bae')
count_p_d_b, lst_p_d_b = comparing(b_s,lst_p_d)
print(count_p_d_b)
print('phishing')
count_p_d_b_phish, lst_p_d_b_phish = comparing(clean_up_list(b_s_phish),lst_p_d_phish)
print(count_p_d_b_phish)
print('legit')
count_p_d_b_legit, lst_p_d_b_legit = comparing(clean_up_list(b_s_legit),lst_p_d_legit)
print(count_p_d_b_legit)

print('************************************')
print('Intersection between textfooler, pwws, deepwordbug and bae')
count_t_p_d_b, lst_t_p_d_b = comparing(b_s,lst_t_p_d)
print(count_t_p_d_b)
print('phishing')
count_t_p_d_b_phish, lst_t_p_d_b_phish = comparing(clean_up_list(b_s_phish),lst_t_p_d_phish)
print(count_t_p_d_b_phish)
print('legit')
count_t_p_d_b_legit, lst_t_p_d_b_legit = comparing(clean_up_list(b_s_legit),lst_t_p_d_legit)
print(count_t_p_d_b_legit)


Intersection between textfooler and pwws
69
phishing
52
legit
17
Intersection between textfooler and deepwordbug
64
phishing
51
legit
13
Intersection between textfooler and bae
20
phishing
7
legit
13
Intersection between pwws and deepwordbug
59
phishing
51
legit
8
Intersection between pwws and bae
13
phishing
7
legit
6
Intersection between deepwordbug and bae
12
phishing
7
legit
5
************************************
Intersection between textfooler, pwws and deepwordbug
59
phishing
51
legit
5
Intersection between textfooler, pwws and bae
13
phishing
7
legit
13
Intersection between textfooler, deepwordbug and bae
12
phishing
7
legit
5
Intersection between pwws, deepwordbug and bae
13
phishing
7
legit
4
************************************
Intersection between textfooler, pwws, deepwordbug and bae
11
phishing
7
legit
5


### Make a list of of purtubed words and their original words

In [15]:
def original_purturbed_word_file(succeeded_examples, attack_examples, attack_name):

    indexs = list(range(0,len(succeeded_examples)))
    original_words = extract_purturb_words(succeeded_examples, indexs)
    purturbed_words = extract_purturb_words(attack_examples, indexs)

    original_words_lst = []
    purturbed_words_lst = []

    for i in range(len(original_words)):
        for j in range(len(original_words[i])):
            original_words_lst.append(original_words[i][j])
            purturbed_words_lst.append(purturbed_words[i][j])
    
    words_df = pd.DataFrame(
    {'Original Word': original_words_lst,
     'Purturbed Word': purturbed_words_lst
    })
    words_df.to_csv(attack_name + '.csv')
    return words_df


In [16]:
#t_s_phish, t_a_phish, t_s_legit, t_a_legit
df_phish_t = original_purturbed_word_file(t_s_phish, t_a_phish, 'phish_textfooler')
df_legit_t = original_purturbed_word_file(t_s_legit, t_a_legit, 'legit_textfooler')
df_phish_p = original_purturbed_word_file(p_s_phish, p_a_phish, 'phish_pwws')
df_legit_p = original_purturbed_word_file(p_s_legit, p_a_legit, 'legit_pwws')
df_phish_d = original_purturbed_word_file(d_s_phish, d_a_phish, 'phish_deepwordbug')
df_legit_d = original_purturbed_word_file(d_s_legit, d_a_legit, 'legit_deepwordbug')
df_phish_b = original_purturbed_word_file(b_s_phish, b_a_phish, 'phish_bae')
df_legit_b = original_purturbed_word_file(b_s_legit, b_a_legit, 'legit_bae')

### Find POS tag of purturbed words in different attack exampls Using stanza

In [17]:
import stanza

In [18]:
# def extract_pos_tags(texts):
#     pos_tags = []
#     for text in texts:
#         text = word_tokenize(text)
#         pos_tags.append(nltk.pos_tag(text))
#     return pos_tags

def extract_pos_tags(texts):
  nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
  pos_tags = []
  for text in texts:
    doc = nlp(text)
    lst = []
    lst_word = []
    for sent in doc.sentences:
      for word in sent.words:
        a = (word.text,word.xpos)
        lst.append(a)
    pos_tags.append(lst)
    
  return pos_tags

# def purturbed_words_pos_tags(attack_examples):
#     purturbed_pos_tag_2 = []
#     word_lst2 = []
#     final_pos_tag_lst = []
    
#     indexs = list(range(0,len(attack_examples)))
#     purturbed_words = extract_purturb_words(attack_examples, indexs)
#     p_pos_tags = extract_pos_tags(clean_up_list(attack_examples))
    
#     for i in range(len(p_pos_tags)):
#         word_lst1 = []
#         purturbed_pos_tag_1 = []
#         for j in range(len(p_pos_tags[i])):
#             if p_pos_tags[i][j][0] in purturbed_words[i]:
#                 word_lst1.append(p_pos_tags[i][j][0])
#                 purturbed_pos_tag_1.append(p_pos_tags[i][j][1])
        
#         word_lst2.append(word_lst1) 
#         purturbed_pos_tag_2.append(purturbed_pos_tag_1)
    
    
#     for i in range(len(purturbed_words)):
#         for j in range(len(purturbed_words[i])):
#             if purturbed_words[i][j] in word_lst2[i]:
#                 index = word_lst2[i].index(purturbed_words[i][j])
#                 final_pos_tag_lst.append(purturbed_pos_tag_2[i][index])
            
#     return word_lst2, purturbed_pos_tag_2, final_pos_tag_lst
        

def pos_tag_evaluation(succeeded,attack):
    indexs = list(range(0,len(succeeded)))
    original_words = extract_purturb_words(succeeded, indexs)
    purturbed_words = extract_purturb_words(attack, indexs)
    s_pos_tags = extract_pos_tags(clean_up_list(succeeded))
    a_pos_tags = extract_pos_tags(clean_up_list(attack))
    count_true = 0
    count_all = 0


    for j in range(len(clean_up_list(attack))):
        #a = word_tokenize(clean_up_list(attack)[j])
        a = []
        for i in range(len(a_pos_tags[j])):
            a.append(a_pos_tags[j][i][0])
        for i in range(len(purturbed_words[j])):
            if purturbed_words[j][i] in a:
                k = a.index(purturbed_words[j][i])
                if k < len(s_pos_tags[j]) and (s_pos_tags[j][k][0] != a_pos_tags[j][k][0]):
                  if s_pos_tags[j][k][1] == a_pos_tags[j][k][1]:
                      count_true += 1
                      count_all += 1
                      # print(s_pos_tags[j][k])
                      # print(a_pos_tags[j][k])
                      # print('****')
                  else:
                      count_all += 1
                      # print(s_pos_tags[j][k])
                      # print(a_pos_tags[j][k])
                      # print('$$$$$')

    return (count_true/count_all)         


POS-Tag Evaluation Results

In [19]:
print('Textfooler Successful Attack POS-Tag Evaluation')
print(pos_tag_evaluation(t_succeeded, t_attack))

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Textfooler Successful Attack POS-Tag Evaluation


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


0.5398981324278438


In [ ]:
print('PWWS Successful Attack POS-Tag Evaluation')
print(pos_tag_evaluation(p_succeeded, p_attack))

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


0.5578034682080925


In [ ]:
print('DeepWordBug Successful Attack POS-Tag Evaluation')
print(pos_tag_evaluation(b_succeeded, b_attack))

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


0.44554455445544555
